# Crowd Counting
by David Ohm

This notebook will step you through the process of training and testing a deep learning model for estimating crowd density. The output heatmap (i.e., density map) can be used to compute an estimate of the total number of peaople in an image. Note - the same model could be trained to count other objects (i.e., vehicles, fruit, microscopic organisms, etc.). 

The model in this notebook is based on the model in the paper [CSRNet: Dilated Convolutional Neural Networks for Understanding the Highly Congested Scenes](https://arxiv.org/abs/1802.10062).

### Results:

TBD   

### Dataset:

ShanghaiTech dataset

### Training Parameters:

1. *Loss* = MSE;

2. *Optimizer* = SGD(lr=1e-6);

3. *Batch size*: 1;

4. *Data augmentation*: Flip horizontally randomly;

5. *Weights*: Got best weights of SHB in epoch xxx, the best one of SHA in epoch xxx   

In [1]:
import os
import cv2
import time
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm as CM
from PIL import Image
import h5py
import math
import glob
import sys
import pathlib

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.keras import metrics
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, multiply, BatchNormalization, ReLU, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json
import tensorflow as tf

if tf.__version__ != '2.0.0':
    tf.compat.v1.enable_v2_behavior()
    tf.compat.v1.enable_eager_execution()

# Scroll down to see TF2 Version

In [ ]:
K.clear_session()
root = pathlib.Path('/data/crowd_counting/ShanghaiTech/')

In [ ]:
part_A_train = root.joinpath('part_A/train_data','images')
part_A_test = root.joinpath('part_A/test_data','images')
part_B_train = root.joinpath('part_B/train_data','images')
part_B_test = root.joinpath('part_B/test_data','images')

temp = 'test_images'
train_path_sets = [part_B_train]
test_path_sets = [part_B_test]

In [ ]:
files = [str(path) for path in part_B_train.glob('*.jpg')]

In [ ]:
plt.rcParams["figure.figsize"] = (12,7)   # set plot sizes
print('File[100] path is: ', files[100])
img = plt.imread(files[100])
plt.imshow(img)

In [ ]:
print('Image shape is: ', img.shape)
print('Image type is: ', img.dtype)
print('min/max size = ', np.min(img), np.max(img))

In [ ]:
# generate training image paths
train_img_paths = []

for path in train_path_sets:
    img_paths = [str(img_path) for img_path in path.glob('*.jpg')]
    train_img_paths.extend(img_paths)
        
print("Total images : ",len(train_img_paths))
print("Example path is: ", train_img_paths[100])

In [ ]:
# generate test image paths
test_img_paths = []

for path in test_path_sets:
    img_paths = [str(img_path) for img_path in path.glob('*.jpg')]
    test_img_paths.extend(img_paths)
        
print("Total images : ",len(test_img_paths))
print("Example path is: ", test_img_paths[100])

In [ ]:
# Setup some variables
num_train_images = len(train_img_paths)
num_test_images = len(test_img_paths)
print('Number of training images: ', num_train_images)
print('Number of test images: ', num_test_images)
batch_size = 1
num_epochs = 100

In [ ]:
@tf.function
def get_input(path):
    #Function to load,normalize and return image 
    im = tf.io.read_file(path)
    im = tf.image.decode_jpeg(im, channels=3)
#     Image.open(path).convert('RGB') 
#     im = np.array(im)
    #print('min/max size = ', np.min(jnk), np.max(jnk))   
    im = tf.cast(im, tf.float32) / 255.0
    
    channels = tf.unstack(im, axis=2)
        
    channels[0]=(channels[0]-0.485)/0.229
    channels[1]=(channels[1]-0.456)/0.224
    channels[2]=(channels[2]-0.406)/0.225
    
    im = tf.stack(channels, axis=2)
    
    #im = im.astype('float32')
    return im

In [ ]:
jnk = get_input(train_img_paths[100])
print('Image shape is: ', jnk.shape)
print('Image type is: ', jnk.dtype)
print('min/max size = ', np.min(jnk[:,:,1]), np.max(jnk[:,:,1]))

In [ ]:
def get_output(path):
    #import target density map and resize target to 1/8 input size since netwrok downsamples to 1/8 input size
    gt_file = h5py.File(path,'r')
    
    target = tf.constant(gt_file['density'], tf.float32)
    ## Note - multiply by 64 due to 1/8 downsample, then multiply by another 64 to boost pixel values in order to train (ignore edge artifacts)
    img = tf.expand_dims(target,axis=2)
    img = tf.image.resize(img,(int(target.shape[0]/8),int(target.shape[1]/8)),method=tf.image.ResizeMethod.BICUBIC)*64*64 
    return img

In [ ]:
# def get_input_hm(path):
#     #import target density map and resize target to 1/8 input size since netwrok downsamples to 1/8 input size
#     gt_file = h5py.File(path,'r')
#     target = np.asarray(gt_file['density'])
#     img = cv2.resize(target,(int(target.shape[1]),int(target.shape[0])),interpolation = cv2.INTER_LINEAR)*1024*64
     
#     img = np.expand_dims(img,axis  = 2) 
#     #img = np.stack((img,)*3, axis=-1)    
#     #print(img.shape)
    
#     return img

In [ ]:
jnk = get_output('/data/crowd_counting/ShanghaiTech/part_B/train_data/ground/IMG_148.h5')
jnk.shape
#jnk = get_input_hm('/data/crowd_counting/ShanghaiTech/part_B/train_data/ground/IMG_148.h5')
#jnk.shape

In [ ]:
# Let's test this last function's calls and look at a sample density plot (i.e. heatmap) and get ground-truth count
test_heatmap_path = train_img_paths[100].replace('.jpg','.h5').replace('images','ground')
test_heatmap_path

In [ ]:
# gt_file = h5py.File(test_heatmap_path,'r')
# target = np.asarray(gt_file['density'])
# img = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64   ## 8 x 8 = 64 
# plt.imshow(img,cmap=CM.jet)
# print(img.shape)
# print("Ground-Truth Sum = " ,np.sum(img))
img = tf.squeeze(get_output(test_heatmap_path)).numpy() / 64
print("Ground-Truth Sum = " ,np.sum(img))
plt.imshow(img,cmap=CM.jet)

In [ ]:
print('Image shape is: ', img.shape)
print('Image type is: ', img.dtype)
print('min/max size = ', np.min(img), np.max(img))

In [ ]:
#def preprocess_input(image,target):
#    #crop image and crop target
#    #resize target
#    crop_size = (int(image.shape[1]/2),int(image.shape[2]/2))
#    crop_size_target = (int(target.shape[1]/2),int(target.shape[2]/2))
    
    
#    if random.randint(0,9)<= 1:            
#            dx = int(random.randint(0,1)*image.shape[1]*1./2)
#            dy = int(random.randint(0,1)*image.shape[2]*1./2)
#            dxt = int(random.randint(0,1)*target.shape[1]*1./2)
#            dyt = int(random.randint(0,1)*target.shape[2]*1./2)
#            
#    else:
#            dx = int(random.random()*image.shape[1]*1./2)
#            dy = int(random.random()*image.shape[2]*1./2)
#            dxt = int(random.random()*target.shape[1]*1./2)
#            dyt = int(random.random()*target.shape[2]*1./2)
#
#    #print(crop_size , dx , dy)
#    img = image[:, dx:crop_size[0]+dx , dy:crop_size[1]+dy, :]
#    target_aug = target[:,dxt:crop_size_target[0]+dxt,dyt:crop_size_target[1]+dyt,:]
#    
#    print(img.shape)
#    print(target_aug.shape)
#
#    return(img,target_aug)

In [ ]:
# def flip_horizontally(x, y):
#     to_flip = np.random.randint(0, 2)
#     if to_flip:
#         #x = np.squeeze(x)
#         #y = np.squeeze(y)
#         #x, y = cv2.flip(x, 1), np.expand_dims(cv2.flip(np.squeeze(y), 1), axis=-1)
#         x = cv2.flip(x, 1)
#         y = cv2.flip(y, 1)
#         # Suppose shape of y is (123, 456, 1), after cv2.flip, shape of y would turn into (123, 456).
#     return x, y

In [ ]:
# plt.imshow(img)

# TF2 Version

### Create a TFRecord
NOTE: THIS TAKES A WHILE (~5 minutes), SKIP THIS IF THE RECORD HAS ALREADY BEEN CREATED

In [2]:
root = pathlib.Path('/data/crowd_counting/ShanghaiTech/')
part_A_train = root.joinpath('part_A/train_data','images')
part_A_test = root.joinpath('part_A/test_data','images')
part_B_train = root.joinpath('part_B/train_data','images')
part_B_test = root.joinpath('part_B/test_data','images')

def get_images_in_path_sets(path_sets):
    all_img_paths = []
    for path_set in path_sets:
        img_paths = [str(img_path) for img_path in path_set.glob('*.jpg')]
        all_img_paths.extend(img_paths)
    return all_img_paths

train_img_paths = get_images_in_path_sets([part_B_train])
print("Total training images: ",len(train_img_paths))
print("Example train path is: ", train_img_paths[100])

test_img_paths = get_images_in_path_sets([part_B_test])
print("Total training images: ",len(test_img_paths))
print("Example test path is:  ", test_img_paths[100])

Total training images:  400
Example train path is:  /data/crowd_counting/ShanghaiTech/part_B/train_data/images/IMG_148.jpg
Total training images:  316
Example test path is:   /data/crowd_counting/ShanghaiTech/part_B/test_data/images/IMG_278.jpg


In [3]:
def _int64_feature(value):
    """Wrapper for insert int64 feature into Example proto."""
    if not isinstance(value, list) and not isinstance(value, np.ndarray):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float_feature(value):
    """Wrapper for insert float features into Example proto."""
    if not isinstance(value, list) and not isinstance(value, np.ndarray):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _bytes_feature(value):
    """Wrapper for insert bytes features into Example proto."""
    if not isinstance(value, list) and not isinstance(value, np.ndarray):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [4]:
def serialize_example(path):
    # Read in image, encode it as a png
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)

    # Read in label and convert it to numpy array
    gt_file = h5py.File(path.replace('.jpg','.h5').replace('images','ground'),'r')
    label = tf.constant(gt_file['density'], tf.float32)
    
    #TFRecords hold 1D arrays, so store a flattened version of the label along with the original shape
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': _bytes_feature(tf.image.encode_png(img, compression=0).numpy()),
                'label': _bytes_feature(tf.io.serialize_tensor(label).numpy()),
            }                          
        )
    )
    
    return example

In [5]:
train_record_path = "train.record"
print("Creating Training Record at {}".format(str(train_record_path)))

# Create the Train TFRecord
writer = tf.io.TFRecordWriter(train_record_path)
for data in train_img_paths:
    example = serialize_example(data)
    writer.write(example.SerializeToString())
writer.close()

Creating Training Record at train.record


In [6]:
test_record_path = "test.record"
print("Creating Test Record at {}".format(str(test_record_path)))

# Create the Test TFRecord
writer = tf.io.TFRecordWriter(test_record_path)
for data in test_img_paths:
    example = serialize_example(data)
    writer.write(example.SerializeToString())
writer.close()

Creating Test Record at test.record


### Define how to preprocess the data

In [7]:
@tf.function
def parse_record(serialized_example):
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),  # one image one record
        'label': tf.io.FixedLenFeature([], tf.string),
    }

    example = tf.io.parse_single_example(serialized_example, features)

    img = tf.io.decode_png(example['image'])
    label = tf.io.parse_tensor(example['label'], out_type=tf.float32)

    img.set_shape([768, 1024, 3])
    label.set_shape([768, 1024])
    
    return img, label

AttributeError: module 'tensorflow' has no attribute 'function'

In [ ]:
@tf.function
def preprocess(img, label):
    
    # Cast the image to float32
    img = tf.cast(img, tf.float32) / 255.0

    # Split the channels appart to apply operations to each channel seperately
    channels = tf.unstack(img, axis=2)
    
    channels[0]=(channels[0]-0.485)/0.229
    channels[1]=(channels[1]-0.456)/0.224
    channels[2]=(channels[2]-0.406)/0.225
    
    # Stack the channels together again to create the image
    img = tf.stack(channels, axis=2)
        
    # tf.image.resize requires a 3 or 4 dimensionals (h,w,c) or (b,h,w,c)
    label = tf.expand_dims(label,axis=2)

    # resize label density map to 1/8 input size since network downsamples to 1/8 input size
    ## Note - multiply by 64 due to 1/8 downsample, then multiply by another 64 to boost pixel values in order to train (ignore edge artifacts)
    label = tf.image.resize(label,(int(label.shape[0]/8),int(label.shape[1]/8)),method=tf.image.ResizeMethod.BICUBIC)*64*64

    do_flip = tf.random.uniform([]) > 0.5
    if do_flip:
        tf.image.flip_left_right(img)
        tf.image.flip_left_right(label)

    print("Image shape: ", img.shape)
    print("Label shape: ", label.shape)
    
    return img, label

### Create training and testing datasets and map preprocessing functions to them  

In [ ]:
train_record_path = "train.record"
train_ds = tf.data.TFRecordDataset(train_record_path)
train_ds = train_ds.map(parse_record, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# train_ds = train_ds.shuffle(32).batch(1).prefetch(1)
train_ds = train_ds.shuffle(32).repeat().batch(4).prefetch(1)

# The repeat() is needed for TF 1.14 because of a bug where the dataset doesn't loop automatically when using the distribute strategy
# When not using distribute strategy (Training on a single GPU) this is not required and is fixed in TF 2.0

test_record_path = "test.record"
test_ds = tf.data.TFRecordDataset(test_record_path)
test_ds = test_ds.map(parse_record, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# test_ds = test_ds.shuffle(32).batch(1).prefetch(1)
test_ds = test_ds.shuffle(32).repeat().batch(4).prefetch(1)

In [ ]:
# View some of the training data
for (img_batch, label_batch) in train_ds.take(1):
    img, label = img_batch[0], tf.squeeze(label_batch[0]) # Index 0 because we are now batched
    
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(img.numpy())
    axarr[1].imshow(label.numpy(),cmap=CM.jet)
    plt.show()
    print("Ground-Truth Sum = " ,np.sum(label.numpy() / 64))
    print('Image shape is: ', img.numpy().shape)
    print('Image type is: ', img.numpy().dtype)
    print('min/max size = ', np.min(img.numpy()), np.max(img.numpy()))

### Define Loss Functions

In [ ]:
@tf.function
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    sub_var = tf.math.square(y_pred - y_true)
    #tf.print('Sub_var: ', sub_var)
    sum_square = tf.math.reduce_sum(sub_var)
    #tf.print('Sum_square: ', sum_square.shape, sum_square)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

### Define Model 

In [ ]:
# Neural network model : VGG + Conv
def CrowdNet():  
            #Variable Input Size
            rows = None
            cols = None
            channels = 3
            
            #Batch Normalisation option
            
            #batch_norm = 0
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            
            
            
            model = Sequential()
            model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, channels), kernel_initializer = init))
            model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
            model.add(MaxPooling2D(strides=2))
            model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
            model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
            model.add(MaxPooling2D(strides=2))
            model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
            model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
            model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
            model.add(MaxPooling2D(strides=2))            
            model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
            model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
            model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                
                
            #Conv2D
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, kernel_initializer = init, padding = 'same', name = 'output'))
        
            front_end = VGG16(weights='imagenet', include_top=False)
            
            weights_front_end = []
            for layer in front_end.layers:
                if 'conv' in layer.name:
                    weights_front_end.append(layer.get_weights())
            
            counter_conv = 0
            for i in range(len(model.layers)):
                if counter_conv >= 13:
                    break
                if 'conv' in model.layers[i].name:
                    model.layers[i].set_weights(weights_front_end[counter_conv])
                    counter_conv += 1
            
            #model = init_weights_vgg(model)
        
            return model

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = CrowdNet()

In [ ]:
with strategy.scope():
    sgd = SGD(lr = 1e-6, decay = (1e-4), momentum = 0.95)
    #adam = Adam(lr = 1e-4, decay = (5*1e-3))
    model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
    #model.compile(optimizer=adam, loss=euclidean_distance_loss, metrics = ['mse'])

In [ ]:
model.summary()

In [ ]:
x = np.random.random((1,768,1024,3)).astype('float32')
#x = np.random.random((1,768,1024,1)).astype('float32')

In [ ]:
model(x).shape

In [ ]:
# Save the model according to the conditions 
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
# model_save_path = '/data/crowd_counting/saved_models/'
model_name = 'crowd_counting_model-dist'
cp_str = "training/checkpoints/"+model_name+"/epoch{epoch:02d}-loss{loss:.2f}-val_loss{val_loss:.2f}.h5"
# checkpoint = ModelCheckpoint(model_save_path+model_name+"_epoch{epoch:02d}-loss{loss:.2f}.h5", monitor='loss', verbose=1, save_best_only=True,  mode='max' )
os.makedirs("training/checkpoints/{}".format(model_name), exist_ok=True)
callbacks = [
    ModelCheckpoint(cp_str),
    TensorBoard("training/logs/{}".format(model_name))
]

In [ ]:
EPOCHS = 100
print("Training...")
H = model.fit(
    train_ds,
    validation_data=test_ds,
    steps_per_epoch = len(train_img_paths) // 4,
    validation_steps = 75,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# plot the training loss and accuracy
N = num_epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
#plt.plot(np.arange(0, N), H.history["mse"], label="train_mse")
#plt.plot(np.arange(0, N), H.history["val_mean_squared_error"], label="val_mse")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
#save_mod(model,"/data/crowd_counting/weights/model_A_weights.h5","/data/crowd_counting/models/Model_A.json")
#model.save('/data/crowd_counting/saved_models/crowd_counting_modelA_2_51epochs.h5')

In [ ]:
# def load_model():
#     # Function to load and return neural network model 
#     json_file = open('/data/crowd_counting/models/Model_B.json', 'r')
#     #json_file = open('/data/crowd_counting/models/ModelA.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     loaded_model = model_from_json(loaded_model_json)
#     loaded_model.load_weights("/data/crowd_counting/weights/model_B_weights.h5")
#     #loaded_model.load_weights("/dml/notebooks/CSRNet-keras/weights/pretrained.h5")
#     return loaded_model

In [ ]:
def create_img_test(path):
    #Function to load,normalize and return image 
    im = Image.open(path)
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    im = np.expand_dims(im, axis = 0)
    return im

In [ ]:
## Try some predictions to view output of model
# test_m = tf.keras.models.load_model("training/checkpoints/crowd_counting_model-dist/epoch60-loss217.69.h5", compile=False)
test_m = tf.keras.models.load_model("training/checkpoints/crowd_counting_model-dist/epoch90-loss204.93.h5", compile=False)
def predict(path):
    #Function to load image,predict heat map, generate count and return (count , image , heat map)
    #model = load_model()
    image = create_img_test(path)
    ans = test_m.predict(image)
    count = np.sum(ans)
    return count,image,ans

In [ ]:
## use a test image to check the prediction (e.g., inference)
plt.rcParams["figure.figsize"] = (10,6)   # set plot sizes
test_image = test_img_paths[31]
# test_image = '/data/crowd_counting/ShanghaiTech/part_B/test_data/images/IMG_145.jpg'
test_heatmap_truth = test_image.replace('.jpg','.h5').replace('images','ground')
## Get the Ground Truth heatmap
gt_file = h5py.File(test_heatmap_truth,'r')
groundtruth = np.asarray(gt_file['density'])
groundtruth = cv2.resize(groundtruth,(int(groundtruth.shape[1]/8),int(groundtruth.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64   ## 8 x 8 = 64
#plt.imshow(groundtruth,cmap=CM.jet)
print("Ground Truth People Count = " ,int(np.sum(groundtruth)))
#print('Ground Truth Heat Map Dimensions: ', groundtruth.shape)
#print('Image type is: ', groundtruth.dtype)
#print('min/max size = ', np.min(groundtruth), np.max(groundtruth))
#plt.show()

# get the input image
count,image,hmap = predict(test_image)
width = image.shape[2]
height = image.shape[1]
channels = image.shape[3]
print('Input Image Path: ', test_image)
#print('Input Image dimensions: ', width, 'x ', height, 'x ', channels)
input_image = Image.open(test_image)
plt.imshow(input_image)
#plt.imshow(image.reshape(height,width,channels))
plt.show()

# predicted heat map info 
hwidth = hmap.shape[2]
hheight = hmap.shape[1]
hchannels = hmap.shape[3]
#print('Output Heat Map Dimensions: ', hwidth, 'x ', hheight, 'x ', hchannels)
print('Estimated People Count = ', int(count) / 64)
plt.imshow(hmap.reshape(hheight,hwidth), cmap = CM.jet )
plt.show()
